# Pandas & NumPy Practical Assessment

**Instructions**
- Use Python 3.10+ with `pandas` and `numpy`.
- You may import additional standard libraries (`re`, `math`, `datetime`, etc.).
- Do not use external datasets or the internet.
- Answer all questions in the provided cells. If a cell is marked `# YOUR CODE HERE`, write your solution there.
- Keep code readable and modular; prefer small helper functions where useful.
- Where asked, **do not change variable names**—autochecks rely on them.
- Timezone for any “today/now” references: Asia/Kolkata.
- Expected total time: 60–90 minutes.

**Coverage**
- DataFrame creation, dtypes, missing data
- Numeric type extraction, coercion, rounding
- Strict time parsing and separation (date vs time vs datetime)
- Column name standardization (e.g., snake_case, prefixes)
- Mapping/`apply`/`map`/`applymap`, vectorization, `np.where`
- GroupBy, aggregations, `transform`
- Pivot
- Joins/merges
- Windows (`rolling`, `expanding`), rolling average & custom windows
- Basic NumPy utilities

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
import re

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 50)

In [ ]:
# Synthetic dataset generator — run this first.
rng = np.random.default_rng(42)

# Orders-like dataset with deliberately messy inputs
n = 200
base_date = pd.Timestamp('2024-12-15')

customers = ['Alice', 'Bob', 'Carlos', 'Diana', 'Esha', 'Fatima', 'Gopal', 'Hiro', 'Isha', 'Jamal']
products = ['Phone', 'Laptop', 'Headphones', 'Mouse', 'Keyboard', 'Monitor', 'Charger', 'Stand']
categories = {'Phone':'Electronics','Laptop':'Electronics','Headphones':'Accessories','Mouse':'Accessories','Keyboard':'Accessories','Monitor':'Electronics','Charger':'Accessories','Stand':'Accessories'}
cities = ['Mumbai','Delhi','Bengaluru','Hyderabad','Chennai','Pune']

dates = base_date + pd.to_timedelta(rng.integers(0, 90, size=n), unit='D')
# Random times (store as strings with some mess)
times = [f"{rng.integers(0,24):02d}:{rng.integers(0,60):02d}:{rng.integers(0,60):02d}" for _ in range(n)]
times = np.where(rng.random(n) < 0.1, [t[:-3] for t in times], times)  # some 'HH:MM' only
times = np.where(rng.random(n) < 0.1, [t.replace(':', '-') for t in times], times)  # some 'HH-MM-SS'

df_raw = pd.DataFrame({
    'Order ID': rng.integers(10000, 20000, size=n),
    'Customer Name  ': rng.choice(customers, size=n),
    ' product ': rng.choice(products, size=n),
    ' Qty ': rng.integers(1, 6, size=n),
    'Unit Price(INR)': rng.integers(2000, 80000, size=n),
    ' Discount% ': np.where(rng.random(n) < 0.15, rng.integers(5, 30, size=n), np.nan),
    'City ': rng.choice(cities, size=n),
    'OrderDate': dates.astype(str),
    'order_time': times,
    'extra_numeric_like': np.where(rng.random(n) < 0.25,
                                   [f"₹{x:,}" for x in rng.integers(100, 5000, size=n)],
                                   rng.integers(100, 5000, size=n).astype(str)),
    'Note': np.where(rng.random(n) < 0.1, None, '')
})

# Introduce some category mismatches
df_raw.loc[rng.integers(0, n, 5), ' product '] = 'Unknown'
df_raw.loc[rng.integers(0, n, 5), ' Discount% '] = '10%'  # string percent

df_raw.head()

## Q1. DataFrame creation & basic cleanup
1. Copy `df_raw` into `orders`, **do not modify** `df_raw`.
2. Strip/standardize column names to **snake_case** (e.g., `"Customer Name  "` → `customer_name`).
3. Ensure dtypes:
   - `order_id`, `qty` → integer
   - `unit_price_inr` → integer
   - `discount_pct` → float in 0-1 (coerce from strings like `"10%"` or `NaN`)
   - `order_date` → `datetime64[ns]` (date only, time set to midnight)
   - `order_time` → `datetime64[ns]` or `timedelta64[ns]` representing time-of-day. **Keep only the time portion.** Hints: `to_datetime`, format variations, `.dt.time` or normalize to a timedelta from midnight.
   - `extra_numeric_like` → integer, coercing from `"₹1,234"` and strings.
4. Create a tidy categorical column `category` based on `product` using the mapping in `categories`; unknowns → `"Other"`.

In [ ]:
# Q1 — YOUR CODE HERE
# Create `orders` from `df_raw` and perform the requested cleanup.
# Ensure you define: orders
# Tips: rename with regex, use .str methods, pd.to_datetime with errors='coerce', pd.to_timedelta, etc.

pass  # TODO

In [ ]:
# Q1 — checks (do not modify)
assert 'orders' in globals(), "orders not defined"
expected_cols = {'order_id','customer_name','product','qty','unit_price_inr','discount_pct','city','order_date','order_time','extra_numeric_like','note','category'}
assert expected_cols.issubset(set(orders.columns)), f"Missing columns: {expected_cols - set(orders.columns)}"
assert pd.api.types.is_integer_dtype(orders['order_id']), "order_id must be integer"
assert pd.api.types.is_integer_dtype(orders['qty']), "qty must be integer"
assert pd.api.types.is_integer_dtype(orders['unit_price_inr']), "unit_price_inr must be integer"
assert pd.api.types.is_float_dtype(orders['discount_pct']), "discount_pct must be float"
assert pd.api.types.is_datetime64_any_dtype(orders['order_date']), "order_date must be datetime"
assert ('timedelta64' in str(orders['order_time'].dtype) or pd.api.types.is_datetime64_any_dtype(orders['order_time'])), "order_time should be time-like"
assert pd.api.types.is_integer_dtype(orders['extra_numeric_like']), "extra_numeric_like must be integer"
assert orders['category'].notna().all(), "category should have values"
print("Q1 checks passed .")

## Q2. Numeric columns & transformations
1. Select only **numeric** columns into `orders_num` using `select_dtypes`.
2. Create `gross_amount = qty * unit_price_inr`.
3. Create `net_amount = gross_amount * (1 - discount_pct.fillna(0))` and round to nearest integer.
4. Add a column `high_value` using **`np.where`**: mark `"High"` if `net_amount >= 50000`, else `"Regular"`.

In [ ]:
# Q2 — YOUR CODE HERE

pass  # TODO

In [ ]:
# Q2 — checks
assert 'orders_num' in globals(), "orders_num not defined"
assert {'gross_amount','net_amount','high_value'}.issubset(set(orders.columns)), "Derived columns missing"
assert set(orders_num.columns).issubset(set(orders.columns)), "orders_num should be subset of orders"
print("Q2 checks passed .")

## Q3. Systematic column names
1. Write a function `to_snake_case(s)` that converts any column name to snake_case (single `_`, lower-case, strip spaces/symbols).
2. Write a function `standardize_columns(df, prefix_map)` that:
   - applies `to_snake_case`,
   - optionally prefixes columns by domain using a `prefix_map` dict (e.g., `{'order': ['order_id','order_date','order_time']}` → `order__order_id`).
3. Apply it to `orders` to produce `orders_std`. Use a prefix map for: `"order"` → [`order_id`, `order_date`, `order_time`, `order_datetime`].

In [ ]:
# Q3 — YOUR CODE HERE

pass  # TODO

In [ ]:
# Q3 — checks
assert 'orders_std' in globals(), "orders_std not defined"
assert any(col.startswith('order__') for col in orders_std.columns if 'order_id' in col or 'order_date' in col), "Expected 'order__' prefixes"
print("Q4 checks passed .")

## Q4. Pivoting & reshaping
1. Create a summary by city and product with total `net_amount` as a **pivot table**: rows=city, columns=product, values=net_amount.
   - Store as `pivot_city_product`.
2. Melt the pivot back into a tidy long form with columns: `city`, `product`, `net_amount`, dropping missing values → `long_city_product`.

In [ ]:
# Q4 — YOUR CODE HERE

pass  # TODO

In [ ]:
# Q4 — checks
assert 'pivot_city_product' in globals(), "pivot_city_product not defined"
assert 'long_city_product' in globals(), "long_city_product not defined"
assert set(long_city_product.columns) == {'city','product','net_amount'}, "long_city_product wrong columns"
print("Q5 checks passed .")

## Q5. GroupBy, transform & rolling
1. Compute total `net_amount` **per customer** and attach to each row using `groupby().transform('sum')` as `customer_total`.
2. Sort data by `customer_name` and `order_datetime`; compute a **7-day rolling average** of `net_amount` per customer (window='7D' on a time-based index/window) → `customer_rolling_avg_7d`.
3. Compute, per city, a **3-row** rolling sum of `qty` (ordered by `order_datetime`) → `city_qty_roll3`.

In [ ]:
# Q5 — YOUR CODE HERE

pass  # TODO

In [ ]:
# Q5 — checks
for c in ['customer_total','customer_rolling_avg_7d','city_qty_roll3']:
    assert c in orders.columns, f"{c} missing"
print("Q6 checks passed .")

## Q6. Joins & functional transforms
1. Build a small `products_master` DataFrame with columns: `product`, `category`, `gst_rate` (in %), `is_active`.
   - Assume `gst_rate`: Electronics=18, Accessories=12, Other=0; `is_active=True` except Unknown product.
2. Left-join it to `orders` on `product` → `orders_enriched`.
3. Using **`np.where`** or **`apply`**, create `gst_amount = net_amount * (gst_rate/100)`; round to 2 decimals.
4. Using **`map`** or **`replace`**, create `city_tier` mapping for cities: Metro (`Mumbai, Delhi, Bengaluru`) else `Non-Metro`.
5. Using **`applymap`** on a small 2x2 demo frame, show how to trim strings and lowercase across all cells.

In [ ]:
# Q6 — YOUR CODE HERE

pass  # TODO

In [ ]:
# Q6 — checks
assert 'orders_enriched' in globals(), "orders_enriched not defined"
assert {'gst_rate','gst_amount','city_tier'}.issubset(orders_enriched.columns), "Enriched columns missing"
print("Q7 checks passed .")

In [ ]:
# End of assessment
print("All tasks attempted. Run the individual checks above to validate your answers.")